In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InR

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-02-05 21:27:54--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.11MB/s    in 0.2s    

2022-02-05 21:27:55 (6.11 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://abassetti-bucket.s3.amazonaws.com/amazon_reviews_us_Grocery_v1_00.tsv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Grocery_v1_00.tsv"), sep="\t", header=True, inferSchema=True)


### Create DataFrames to match tables

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   42521656|R26MV8D0KG6QI6|B000SAQCWC|     159713740|The Cravings Plac...|         Grocery|          5|            0|          0|   N|                Y|Using these for y...|As a family aller...| 2015-08-31|
|         US|   12049833|R1OF8GP57AQ1A0|B00509LVIQ|     138680402|Mauna Loa Macadam...|         Grocery|          5|    

In [6]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R26MV8D0KG6QI6|          5|            0|          0|   N|                Y|
|R1OF8GP57AQ1A0|          5|            0|          0|   N|                Y|
|R3VDC1QB6MC4ZZ|          5|            0|          0|   N|                N|
|R12FA3DCF8F9ER|          5|            0|          0|   N|                Y|
| RTWHVNV6X4CNJ|          5|            0|          0|   N|                Y|
| RIG9AWFOGRDVO|          2|            1|          1|   N|                Y|
|R1S1XSG4ZCHDGS|          5|            1|          1|   N|                Y|
| RB15NBVY5ELVW|          5|            2|          2|   N|                Y|
| R56358YM1ZJ7I|          5|            0|          0|   N|                N|
|R1ODXB3C9UP3NL|          1|            1|          3|   N|     

In [7]:
total_votes_df = vine_df.filter(vine_df.total_votes >= 20)
total_votes_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1OAZUG90XPU2U|          5|          250|        274|   N|                Y|
|R23VPTUJ8I8NR5|          1|           28|         30|   N|                Y|
|R33VIWRD2X5IA9|          1|            0|         20|   N|                Y|
| R6SRJFJ5YH4UM|          5|           23|         26|   N|                Y|
|R3DOS07BIMP4DJ|          2|           32|         42|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [9]:
# Filter DataFrame for helpful_votes ratio above or equal to 50%
helpful_votes_df = total_votes_df.filter((total_votes_df.helpful_votes / total_votes_df.total_votes) >= 0.5)
helpful_votes_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1OAZUG90XPU2U|          5|          250|        274|   N|                Y|
|R23VPTUJ8I8NR5|          1|           28|         30|   N|                Y|
| R6SRJFJ5YH4UM|          5|           23|         26|   N|                Y|
|R3DOS07BIMP4DJ|          2|           32|         42|   N|                N|
|R30PGWX4HN1CFG|          1|           85|         95|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [10]:
# Create paid vine DataFrame
paid_vine_df = helpful_votes_df.filter(helpful_votes_df.vine == 'Y')
paid_vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3MKO875WGIEBJ|          5|           34|         37|   Y|                N|
|R3OPNTK61FQ7MP|          5|           32|         37|   Y|                N|
| RMDRU8I773X5U|          4|           22|         25|   Y|                N|
| RR83RGWFEFZCL|          3|           61|         64|   Y|                N|
|R2TMZ7GV8SPIRV|          4|           23|         23|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [11]:
# Create paid vine DataFrame
unpaid_vine_df = helpful_votes_df.filter(helpful_votes_df.vine == 'N')
unpaid_vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1OAZUG90XPU2U|          5|          250|        274|   N|                Y|
|R23VPTUJ8I8NR5|          1|           28|         30|   N|                Y|
| R6SRJFJ5YH4UM|          5|           23|         26|   N|                Y|
|R3DOS07BIMP4DJ|          2|           32|         42|   N|                N|
|R30PGWX4HN1CFG|          1|           85|         95|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [14]:
# paid total number of reviews
total_paid_reviews = paid_vine_df.count()
total_paid_reviews

61

In [15]:
# paid 5-star reviews
paid_five_star_reviews = paid_vine_df.filter(paid_vine_df.star_rating == 5).count()
paid_five_star_reviews


20

In [16]:
# paid 5-star reviews percentage
paid_five_star_percent = (paid_five_star_reviews / total_paid_reviews) * 100
paid_five_star_percent

32.78688524590164

In [17]:
# paid total number of reviews
total_unpaid_reviews = unpaid_vine_df.count()
total_unpaid_reviews


28287

In [18]:
# paid 5-star reviews
unpaid_five_star_reviews = unpaid_vine_df.filter(unpaid_vine_df.star_rating == 5).count()
unpaid_five_star_reviews


15689

In [19]:
# paid 5-star reviews percentage
unpaid_five_star_percent = (unpaid_five_star_reviews / total_unpaid_reviews) * 100
unpaid_five_star_percent

55.463640541591545